In [ ]:
#Import Dataset

In [128]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import numpy as np
import smtplib

In [132]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price'}).string.strip()
    except AttributeError:
        try:
            price=soup.find("span", attrs={'class':'a-offscreen'}).string.strip()
        except:
            price=""
                                 
                                                                
       
            
                                                                           
               
                                                                           
        
    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star-small a-star-small-3-5 aok-align-bottom'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [133]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'https://explore.whatismybrowser.com/useragents/parse/?analyse-my-user-agent=yes', 'Accept-Language': 'a-m-us a-aui_72554-c a-aui_accordion_a11y_role_354025-t1 a-aui_killswitch_csa_logger_372963-c a-aui_pci_risk_banner_210084-c a-aui_preload_261698-c a-aui_rel_noreferrer_noopener_309527-c a-aui_template_weblab_cache_333406-c a-aui_tnr_v2_180836-c a-meter-animate'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=skincare+products&crid=2T7N58012O04A&sprefix=%2Caps%2C195&ref=nb_sb_ss_recent_1_0_recent"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [134]:
amazon_df

,title,price,rating,reviews,availability
0,"Simple Daily Skin Detox Purifying Facial Wash,...",$7.98,4.2 out of 5 stars,389 ratings,Not Available
1,Simple Kind to Skin Refreshing Facial Wash Gel...,Page 1 of 1,4.6 out of 5 stars,"11,336 ratings",
10,The Face Shop Rice Water Bright Light Face Cle...,$12.00,4.5 out of 5 stars,"15,154 ratings",In Stock
20,Fixderma Nigrifix Cream 50gm,Page 1 of 1,3.7 out of 5 stars,"2,801 ratings",Currently unavailable.
23,"Glenmark La Shield Sunscreen Gel SPF 40, PA+++...",Page 1 of 1,4.4 out of 5 stars,"4,508 ratings",Currently unavailable.
33,Foxtale SkinKit | The Daily Duet Cleanser Face...,Page 1 of 1,4.1 out of 5 stars,23 ratings,Currently unavailable.
